In [10]:
from pyspark.sql import SparkSession
spark = SparkSession \
 .builder \
 .appName("Lake Dataset Project") \
 .config("spark.some.config.option", "some-value") \
 .getOrCreate()
sc = spark.sparkContext
from pyspark.sql.types import *
sc = spark.sparkContext
lake_df = sc.textFile("./Data/Lake data/mces_lakes_1999_2014.txt")

In [13]:
#filter out the header, make sure the rest looks correct
header = lake_df.first()
log_rows = lake_df.filter(lambda line: line != header)
temp_var = log_rows.map(lambda k: k.split("\t"))
df      =    temp_var.toDF(header.split("\t"))
df.collect()

[Row(PROJECT_ID='7108', DATA_SET_TITLE='Citizen Assisted Monitoring Program (CAMP) for Lakes', LAKE_NAME='Acorn Lake', CITY='Oakdale', COUNTY='Washington', DNR_ID_Site_Number='82010200-01', MAJOR_WATERSHED='Lower St. Croix River', WATER_PLANNING_AUTHORITY='Valley Branch WD', LAKE_SITE_NUMBER='1', START_DATE='2006-04-16', START_HOURMIN24='0:00', END_DATE='2006-04-16', END_HOURMIN24='0:00', SAMPLE_DEPTH_IN_METERS='0', Seasonal_Lake_Grade_RESULT='', Seasonal_Lake_Grade_QUALIFIER='', Seasonal_Lake_Grade_Units='0-4 Categorical Calculated Seasonally: 4 good & 0 bad', Physical_Condition_RESULT='1', Physical_Condition_QUALIFIER='Approved', Physical_Condition_Units='1-5 Categorical: 1 good & 5 bad', Recreational_Suitability_RESULT='5', Recreational_Suitability_QUALIFIER='Approved', Recreational_Suitability_Units='1-5 Categorical: 1 good & 5 bad', Secchi_Depth_RESULT_SIGN='', Secchi_Depth_RESULT='1', Secchi_Depth_QUALIFIER='Approved', Secchi_Depth_Units='m', Total_Phosphorus_RESULT_SIGN='', Tota

In [12]:
records = df.map(tuple)

AttributeError: 'DataFrame' object has no attribute 'map'

In [56]:
monitor = sc.textFile('./Data/Parcel_Lake_Monitoring_Site_Xref.txt') \
            .map(lambda k: k.split("\t"))
header_mon = monitor.first()

Py4JJavaError: An error occurred while calling o730.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/qe5106hi/Desktop/lake_project/Data/references/Parcel_Lake_Monitoring_Site_Xref.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfu

In [ ]:
unique_sites = df.select("DNR_ID_Site_Number").distinct().intersect(monitor.select("Monit_MAP_CODE1").distinct())              


In [50]:
lake_code = df.select(["DNR_ID_Site_Number","LAKE_NAME"])\
                .groupby("LAKE_NAME","DNR_ID_Site_Number")\
                .agg({"DNR_ID_Site_Number":'count'}) \
                .drop("count(DNR_ID_Site_Number)")
lake_code.show()

+--------------------+------------------+
|           LAKE_NAME|DNR_ID_Site_Number|
+--------------------+------------------+
|         Jackson WMA|       82030500-01|
|    Big Comfort Lake|       13005300-01|
|        Calhoun Lake|       27003100-01|
|Fireman's Clayhol...|       10022600-01|
|        Peltier Lake|       02000400-01|
|           Bone Lake|       82005400-01|
|     Courthouse Lake|       10000500-01|
|          Karth Lake|       62007200-01|
|         Lake Forest|       62018700-01|
|           Ryan Lake|       27005800-01|
|       Woodpile Lake|       82013200-01|
|        Diamond Lake|       27012500-01|
|          Downs Lake|       82011000-01|
|           Pike Lake|       62006900-01|
|           Bass Lake|       82003500-01|
|      Halfbreed Lake|       82008000-01|
|     White Bear Lake|       82016700-01|
|   Battle Creek Lake|       82009100-01|
|   Lake of the Isles|       27004000-01|
|        Schultz Lake|       19007500-01|
+--------------------+------------

In [6]:
class Pipeline:
    def __init__(self,filePath):
        self.input_ = self.read_text(filePath)
        self.tasks = []
    
    def read_text(self,filePath):
        lake_df = sc.textFile(filePath)
        header = lake_df.first()
        log_rows = lake_df.filter(lambda line: line != header)
        temp_var = log_rows.map(lambda k: k.split("\t"))
        df = temp_var.toDF(header.split("\t"))
        return df
        
    def task(self, after = None):
        idx = 0
        if after:
            idx = self.tasks.index(after) + 1
        def inner(f):
            self.tasks.insert(idx, f)
            return f
        return inner
    
    def run(self):
        output = self.input_
        for task in self.tasks:
            output = task(output)
        return output

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession \
 .builder \
 .appName("Lake Dataset Project") \
 .config("spark.some.config.option", "some-value") \
 .getOrCreate()
sc = spark.sparkContext




    
lake_pipeline = Pipeline(filePath = './Data/mces_lakes_1999_2014.txt')

@lake_pipeline.task()
def lake_code(df):
    lake_code = df.select(["DNR_ID_Site_Number","LAKE_NAME"])\
                .groupby("LAKE_NAME","DNR_ID_Site_Number")\
                .agg({"DNR_ID_Site_Number":'count'}) \
                .drop("count(DNR_ID_Site_Number)")
    return lake_code

@lake_pipeline.task(after  = lake_code)
def create_database():
    conn = psycopg2.connect("dbname=minnesotaLake user=mnLake")
    cur = conn.cursor()
    cur.execute("DROP TABLE IF EXISTS lakes")
    cur.execute(""" 
    """)

    conn.commit()
    conn.close()


lake_code = lake_pipeline.run()

In [10]:
lake_code.show()

+--------------------+------------------+
|           LAKE_NAME|DNR_ID_Site_Number|
+--------------------+------------------+
|         Jackson WMA|       82030500-01|
|    Big Comfort Lake|       13005300-01|
|        Calhoun Lake|       27003100-01|
|Fireman's Clayhol...|       10022600-01|
|        Peltier Lake|       02000400-01|
|           Bone Lake|       82005400-01|
|     Courthouse Lake|       10000500-01|
|          Karth Lake|       62007200-01|
|         Lake Forest|       62018700-01|
|           Ryan Lake|       27005800-01|
|       Woodpile Lake|       82013200-01|
|        Diamond Lake|       27012500-01|
|          Downs Lake|       82011000-01|
|           Pike Lake|       62006900-01|
|           Bass Lake|       82003500-01|
|      Halfbreed Lake|       82008000-01|
|     White Bear Lake|       82016700-01|
|   Battle Creek Lake|       82009100-01|
|   Lake of the Isles|       27004000-01|
|        Schultz Lake|       19007500-01|
+--------------------+------------

In [2]:
from queries import *



In [3]:

cur.execute(drop_lakes_table)
cur.execute(create_lakes_table)

NameError: name 'cur' is not defined

In [ ]:
"""PROJECT_ID:  INTEGER PRIMARY KEY,
DATA_SET_TITLE  VARCHAR(30),
LAKE_NAME  VARCHAR(30),
CITY  VARCHAR(30),
COUNTY  VARCHAR(30),
DNR_ID_Site_Number  VARCHAR(30),
MAJOR_WATERSHED  VARCHAR(30),
WATER_PLANNING_AUTHORITY  VARCHAR(30),
LAKE_SITE_NUMBER  VARCHAR(30),
START_DATE  DATE,
START_HOURMIN24  VARCHAR(30),
END_DATE  DATE,
END_HOURMIN24  VARCHAR(30),
SAMPLE_DEPTH_IN_METERS  INTEGER,
Seasonal_Lake_Grade_RESULT  VARCHAR(30),
Seasonal_Lake_Grade_QUALIFIER  VARCHAR(30),
Seasonal_Lake_Grade_Units  VARCHAR(30),
Physical_Condition_RESULT  VARCHAR(30),
Physical_Condition_QUALIFIER  VARCHAR(30),
Physical_Condition_Units  VARCHAR(30),
Recreational_Suitability_RESULT  VARCHAR(30),
Recreational_Suitability_QUALIFIER  VARCHAR(30),
Recreational_Suitability_Units  VARCHAR(30),
Secchi_Depth_RESULT_SIGN  VARCHAR(30),
Secchi_Depth_RESULT  VARCHAR(30),
Secchi_Depth_QUALIFIER  VARCHAR(30),
Secchi_Depth_Units  VARCHAR(30),
Total_Phosphorus_RESULT_SIGN  VARCHAR(30),
Total_Phosphorus_RESULT  VARCHAR(30),
Total_Phosphorus_QUALIFIER  VARCHAR(30),
Total_Phosphorus_Units  VARCHAR(30),
longitude  VARCHAR(30),
latitude  VARCHAR(30),"""

In [25]:
class DAG:
    def __init__(self):
        self.graph = {}
    
    def add(self, node, to=None):
        if not node in self.graph:
                self.graph[node] = []
        if to:
            if not to in self.graph:
                self.graph[to] = []
            self.graph[node].append(to)
                

dag = DAG()
dag.add("Load Data")
dag.add("Load Data", "lake code table")
dag.add("Load Data", "Write to Database")
dag.add("lake code table", "find unique")


print(dag.graph)

{'Load Data': ['lake code table', 'Write to Database'], 'lake code table': ['find unique'], 'Write to Database': [], 'find unique': []}


In [42]:
class DATA_RDD:
    def __init__(self,data,inDatabase):
        self.data = data
        self.inDatabase = False
    def print(self):
        print("in Database = ",self.inDatabase)
        

In [43]:
parcel = DATA_RDD("df",False)

In [94]:
conn = psycopg2.connect("dbname=mn_lakes user=kapil")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS lakes")
cur.execute(""" 
CREATE TABLE lakes(
PROJECT_ID  INTEGER PRIMARY KEY,
DATA_SET_TITLE  VARCHAR(100),
LAKE_NAME  VARCHAR(100),
CITY  VARCHAR(100),
COUNTY  VARCHAR(100),
DNR_ID_Site_Number  VARCHAR(100),
MAJOR_WATERSHED  VARCHAR(100),
WATER_PLANNING_AUTHORITY  VARCHAR(100),
LAKE_SITE_NUMBER  VARCHAR(100),
START_DATE  VARCHAR(100),
START_HOURMIN24  VARCHAR(100),
END_DATE  VARCHAR(100),
END_HOURMIN24  VARCHAR(100),
SAMPLE_DEPTH_IN_METERS  INTEGER,
Seasonal_Lake_Grade_RESULT  VARCHAR(100),
Seasonal_Lake_Grade_QUALIFIER  VARCHAR(100),
Seasonal_Lake_Grade_Units  VARCHAR(100),
Physical_Condition_RESULT  VARCHAR(100),
Physical_Condition_QUALIFIER  VARCHAR(100),
Physical_Condition_Units  VARCHAR(100),
Recreational_Suitability_RESULT  VARCHAR(100),
Recreational_Suitability_QUALIFIER  VARCHAR(100),
Recreational_Suitability_Units  VARCHAR(100),
Secchi_Depth_RESULT_SIGN  VARCHAR(100),
Secchi_Depth_RESULT  VARCHAR(100),
Secchi_Depth_QUALIFIER  VARCHAR(100),
Secchi_Depth_Units  VARCHAR(100),
Total_Phosphorus_RESULT_SIGN  VARCHAR(100),
Total_Phosphorus_RESULT  VARCHAR(100),
Total_Phosphorus_QUALIFIER  VARCHAR(100),
Total_Phosphorus_Units  VARCHAR(100),
longitude  VARCHAR(100),
latitude  VARCHAR(100)
)
""")

conn.commit()


In [95]:
print( conn.get_dsn_parameters(),"\n")

{'user': 'kapil', 'passfile': '/Users/qe5106hi/.pgpass', 'dbname': 'mn_lakes', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '1', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [55]:
cur.execute("select * from lakes")

In [96]:
postgres_insert_query =  """INSERT INTO lakes (PROJECT_ID, DATA_SET_TITLE, LAKE_NAME, CITY, COUNTY,
    DNR_ID_Site_Number, MAJOR_WATERSHED, WATER_PLANNING_AUTHORITY, LAKE_SITE_NUMBER,
    START_DATE, START_HOURMIN24, END_DATE, END_HOURMIN24, SAMPLE_DEPTH_IN_METERS, 
    Seasonal_Lake_Grade_RESULT, Seasonal_Lake_Grade_QUALIFIER, Seasonal_Lake_Grade_Units,
    Physical_Condition_RESULT, Physical_Condition_QUALIFIER, Physical_Condition_Units, 
    Recreational_Suitability_RESULT, Recreational_Suitability_QUALIFIER, Recreational_Suitability_Units, 
    Secchi_Depth_RESULT_SIGN, Secchi_Depth_RESULT, Secchi_Depth_QUALIFIER, Secchi_Depth_Units, 
    Total_Phosphorus_RESULT_SIGN, Total_Phosphorus_RESULT, Total_Phosphorus_QUALIFIER, 
    Total_Phosphorus_Units, longitude, latitude)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    
    

In [97]:

record_to_insert = tuple(k.asDict().values())
cur.execute(postgres_insert_query, record_to_insert)
conn.commit()

In [75]:
m = ['PROJECT_ID', 'DATA_SET_TITLE', 'LAKE_NAME', 'CITY', 'COUNTY', 'DNR_ID_Site_Number', 'MAJOR_WATERSHED', 'WATER_PLANNING_AUTHORITY', 'LAKE_SITE_NUMBER', 'START_DATE', 'START_HOURMIN24', 'END_DATE', 'END_HOURMIN24', 'SAMPLE_DEPTH_IN_METERS', 'Seasonal_Lake_Grade_RESULT', 'Seasonal_Lake_Grade_QUALIFIER', 'Seasonal_Lake_Grade_Units', 'Physical_Condition_RESULT', 'Physical_Condition_QUALIFIER', 'Physical_Condition_Units', 'Recreational_Suitability_RESULT', 'Recreational_Suitability_QUALIFIER', 'Recreational_Suitability_Units', 'Secchi_Depth_RESULT_SIGN', 'Secchi_Depth_RESULT', 'Secchi_Depth_QUALIFIER', 'Secchi_Depth_Units', 'Total_Phosphorus_RESULT_SIGN', 'Total_Phosphorus_RESULT', 'Total_Phosphorus_QUALIFIER', 'Total_Phosphorus_Units', 'longitude', 'latitude']
len(m)

33

In [101]:
count = cur.rowcount
print(count, "Record inserted successfully into mobile table")

1 Record inserted successfully into mobile table
